In [8]:
# 모듈 로딩
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
# 저장 경로
SAVE_PATH = '../models/weather/Reg/'
# 모델 구조 및 파라미터 모두 저장 파일명
SAVE_MODEL=SAVE_PATH+'loss(1.58337)_score(0.88159).pth'

In [9]:
class DynamicModel(nn.Module):

    # 모델 구조 설계 함수 즉, 생성자 메서드
    def __init__(self, in_in, in_out, out_out, h_ins=[], h_outs=[]):
        super().__init__()
        
        self.in_layer=nn.Linear(in_in, h_ins[0] if len(h_ins) else in_out )
        
        self.h_layers=nn.ModuleList()
        for idx  in range(len(h_ins)):
            self.h_layers.append( nn.Linear(h_ins[idx], h_outs[idx])  )
        
        self.out_layer=nn.Linear(h_outs[-1]  if len(h_outs) else in_out, out_out)
        
        
    # 학습 진행 콜백 메서드
    def forward(self, x): 
        # 입력층
        y=self.in_layer(x)                  # y=x1w1+x2w2+x3w3+b
        y=F.leaky_relu(y)                         # 0 <= y
        #y=F.relu(self.in_layer(x) )

        # 은닉층
        for linear in self.h_layers:
            y=linear(y)
            y=F.leaky_relu(y) 
            #y=F.relu( linear(y) )
            
        # 출력층
        return self.out_layer(y)       
        

In [10]:
summer_Model = torch.load(SAVE_MODEL,weights_only=False)

[3] 예측 <hr>

In [11]:
data = [float(x) for x in input('평균기온(°C),평균 지면온도(°C),최저기온(°C),평균 이슬점온도(°C) : ').split(',')]

In [12]:
dataTS=torch.FloatTensor(data).reshape(1,-1)
dataTS.shape, dataTS

(torch.Size([1, 4]), tensor([[15.3000, 21.7000,  7.3000,  5.1000]]))

In [13]:
# 새로운 데이터에 대한 예측 즉, predict
summer_Model.eval()
with torch.no_grad():

    # 추론/평가
    pre_val = summer_Model(dataTS)

In [14]:
print('기온은 {:.1f}°C 입니다'.format(pre_val.item()))

기온은 23.0°C 입니다
